<a href="https://colab.research.google.com/github/OlgaChechekina/HW1/blob/master/OlyaCupy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import cupy as np
!nvidia-smi

In [0]:
def needleman_wunsch(x, y, match=1, mismatch=-1, gap=-1):

    weight = np.zeros((len(x) + 1, len(y) + 1), dtype=int)
    direction = np.zeros_like(weight, dtype=int)
    
    weight[:, 0] = np.arange(len(x) + 1) * gap
    weight[0] = np.arange(len(y) + 1) * gap
    
    direction_type = np.array([[0, -1, -1], # diag -- 0
                               [0, -1, 0], # vertical -- 1
                               [0, 0, -1]]) # horizontal -- 2
    
    direction[1:, 0] = 1
    direction[0, 1:] = 2
    
    for i in range(1, len(x) + 1):
        for j in range(1, len(y) + 1):
            direction_type[0, 0] = weight[i-1, j-1] + (match if x[i-1] == y[j-1] else mismatch)
            direction_type[1, 0] = weight[i-1, j] + gap
            direction_type[2, 0] = weight[i, j-1] + gap
#            print(direction_type)
            weight[i, j] = np.max(direction_type[:, 0])
            direction[i, j] = np.argmax(direction_type[:, 0])
            
#            print(weight)
    
    x_res, y_res, i, j = '', '', len(x), len(y)
    while i != 0 or j != 0:
        diff_i, diff_j = direction_type[direction[i, j], 1:]
        i += diff_i
        j += diff_j
        i=int(i)
        j=int(j)
        if diff_i == diff_j:
            x_res += x[i]
            y_res += y[j]
        elif diff_i == -1:
            x_res += x[i]
            y_res += '-'
        else:
            x_res += '-'
            y_res += y[j]

    return int(weight[-1, -1]), x_res[::-1], y_res[::-1]

In [0]:
needleman_wunsch('a', 'b')

(-1, 'a', 'b')

In [0]:
needleman_wunsch('a', 'b', mismatch=-3)

(-2, '-a', 'b-')

In [0]:
needleman_wunsch('aa', 'b')

In [0]:
needleman_wunsch('aaaa', 'b', mismatch=-3)

(-5, '-aaaa', 'b----')

In [0]:
needleman_wunsch('aaaaa', 'bbbbb', gap=2)

(20, '-----aaaaa', 'bbbbb-----')

In [0]:
needleman_wunsch('tccCAGTTATGTCAGgggacacgagcatgcagagac'.upper(), 'aattgccgccgtcgttttcagCAGTTATGTCAGatc'.upper(),
                mismatch=-3)

In [0]:
needleman_wunsch("gggctgccaa", "ggtcgaa", match=1, mismatch=-1, gap=-1)


(2, 'gggctgccaa', '-gg-t-cgaa')